In [31]:
from dataclasses import dataclass
import pulp
import pandas as pd
import numpy as np

@dataclass
class Armor:
    poise: int
    weight: float
    name: str
    location: str = None
    _var = None

    @property
    def var(self):
        if self._var is None:
            self._var = pulp.LpVariable(self.name, 0, 1, pulp.LpInteger)
        return self._var


@dataclass
class ArmorSet:
    poise: int = 0
    weight: float = 0
    armor: Armor = None
    gauntlet: Armor = None
    greave: Armor = None



def get_poise(armors):
    return [armor.poise * armor.var for armor in armors]

def get_weight(armors):
    return [armor.weight * armor.var for armor in armors]

def numeric_coercions(df, column):
    return df[column].apply(lambda x: x if x.isnumeric() else 0.0)
    

def get_armors(armor_type):
    df = pd.read_pickle(f"{armor_type}.pkl")
    df["Poise"] = pd.to_numeric(df["Poise"], errors='coerce')
    df["Wgt."] = pd.to_numeric(df["Wgt."], errors='coerce')
    df.fillna(0, inplace=True)
    return [Armor(name = armor["Name"], poise=armor["Poise"], weight=armor["Wgt."], location=armor["How to Acquire"] ) for armor in df.to_dict(orient='records')]



In [32]:
weight_limit = 21.6

prob = pulp.LpProblem("myProblem", pulp.LpMaximize)

armor_types = ["head", "chest", "arms", "legs"]

typed_armor_lists = [get_armors(armor_type) for armor_type in armor_types]
armors = sum(typed_armor_lists, [])

# Only one of each piece
for armor_list in typed_armor_lists:
    prob += sum([a.var for a in armor_list]) <= 1.0

# Maximum Weigth
weight = sum(get_weight(armors))
prob += weight <= weight_limit

poise = sum(get_poise(armors))
prob += poise


status = prob.solve()

print("| Name | Poise|  Weight |")
print("|:-----|------:|------:|")

for a in armors:
    if pulp.value(a.var) > 0:
        print("|", a.name, "|", a.poise, "|", a.weight, "|")

print("| Total | ", pulp.value(poise), "|", pulp.value(weight), "|")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3q/_dxdmqw96_x981j828nlpw9r0000gn/T/4fa13a90ceb140a7bacfcdc5668bd3e0-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3q/_dxdmqw96_x981j828nlpw9r0000gn/T/4fa13a90ceb140a7bacfcdc5668bd3e0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 1251 RHS
At line 1257 BOUNDS
At line 1510 ENDATA
Problem MODEL has 5 rows, 252 columns and 503 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 42.4127 - 0.00 seconds
Cgl0008I 4 inequality constraints converted to equality constraints
Cgl0005I 4 SOS with 84 members
Cgl0004I processed model has 5 rows, 84 columns (84 integer (84 of which binary)) and 164 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I In